### Asyncio - 비동기 I/O
- asyncio는 async/await 구문을 사용하여 동시성 코드를 작성하는 라이브러리
- 따로 설치를 해줘야 한다. (pip install asyncio)


※ 쓰레드 단점: 디버깅 어렵고, 자원 접근 시 Race Condition(경쟁 상태), 데드락 (Dead Rock) 고려해서 설계 해야 한다.  
※ 코루틴 장, 단점: 하나의 루틴만 실행, 락 관리 필요 X, 제어권으로 실행. 단, 사용 함수가 비동기로 구현되어 있어야 하거나, 직접 비동기로 구현해야 한다.

========== 이하 강의 자료 ==========

- 아래는 코드 패턴이므로 외워서 사용하면 좋다.

### 실습1

In [1]:
import asyncio
import timeit
from urllib.request import urlopen # urlopen은 Block 함수!! 단점
from concurrent.futures import ThreadPoolExecutor
import threading

# 실행 시작 시간
start = timeit.default_timer()
# 서비스 방향이 비슷한 사이트로 실습 권장(예 : 게시판성 커뮤니티)
urls = ['http://daum.net', 'https://naver.com', 'http://mlbpark.donga.com/', 'https://tistory.com', 'https://wemakeprice.com/']

async def fetch(url, executor):
    # 쓰레드명 출력
    print('Thread Name: ', threading.current_thread().getName(), 'Start', url)
    
    # 실행
    res = await loop.run_in_executor(executor, urlopen, url)

    # 종료 순서는 조금씩 달라질 수 있다.
    print('Thread Name: ', threading.current_thread().getName(), 'Done', url)
    
    # 결과 반환
    return res.read()[0:5]

async def main():
    executor = ThreadPoolExecutor(max_workers=6)

    # future 객체 모아서 gather 에서 실행
    futures = [
        asyncio.ensure_future(fetch(url, executor)) for url in urls
    ]

    # 결과 취합
    rst = await asyncio.gather(*futures)

    print()
    print('Result: ', rst)

if __name__ == '__main__':
    # 루프 초기화
    loop = asyncio.get_event_loop()
    # 작업 완료 까지 대기
    loop.run_until_complete(main())
    # 수행 시간 계산
    duration = timeit.default_timer() - start
    # 총 실행 시간
    print('Total Running Time: ', duration)


RuntimeError: This event loop is already running

### 실습2

In [ ]:
import asyncio
import timeit
from urllib.request import urlopen # urlopen은 Block 함수!! 단점
from concurrent.futures import ThreadPoolExecutor
import threading

from bs4 import BeautifulSoup

import sys, io
sys.stdout = io.TextIOWrapper(sys.stdout.detach(), encoding = 'utf-8')
sys.stderr = io.TextIOWrapper(sys.stderr.detach(), encoding = 'utf-8')


# 실행 시작 시간
start = timeit.default_timer()
# 서비스 방향이 비슷한 사이트로 실습 권장(예 : 게시판성 커뮤니티)
urls = ['http://daum.net', 'https://naver.com', 'http://mlbpark.donga.com/', 'https://tistory.com', 'https://wemakeprice.com/']

async def fetch(url, executor):
    # 쓰레드명 출력
    print('Thread Name: ', threading.current_thread().getName(), 'Start', url)
    
    # 실행
    res = await loop.run_in_executor(executor, urlopen, url)

    soup = BeautifulSoup(res.read(), 'html.parser')

    # 전체 페이지 소스 확인
    # print(soup.prettify())
    # 이 부분에서 BeautifulSoup Selector(선택자)를 활용해서 다양한 정보 가져오기 가능
    # 현 예제에서는 페이지 타이틀 정보 수집
    result_data = soup.title

    # 종료 순서는 조금씩 달라질 수 있다.
    print('Thread Name: ', threading.current_thread().getName(), 'Done', url)
    
    # 결과 반환
    return result_data

async def main():
    executor = ThreadPoolExecutor(max_workers=6)

    # future 객체 모아서 gather 에서 실행
    futures = [
        asyncio.ensure_future(fetch(url, executor)) for url in urls
    ]

    # 결과 취합
    rst = await asyncio.gather(*futures)

    print()
    print('Result: ', rst)

if __name__ == '__main__':
    # 루프 초기화
    loop = asyncio.get_event_loop()
    # 작업 완료 까지 대기
    loop.run_until_complete(main())
    # 수행 시간 계산
    duration = timeit.default_timer() - start
    # 총 실행 시간
    print('Total Running Time: ', duration)
